## GeoAI Agricultural Plastic Cover Mapping with Satellite Imagery

This project aims to develop accurate and cost-effective classification models for agricultural plastic cover using freely available satellite datasets. The goal is to advance the mission of global agricultural plastic cover mapping using remote sensing data.

#### GEE

https://code.earthengine.google.com/6503d98eb855b0e50c317370c8cc24e1 

**It is notable that the training samples we are providing are collected by visually implement of satellite in April and May of each test region.**

Features in the training samples are prepared with images on 20240416 for VNM and Spain and on 20240501 for Kenya.


In [1]:
# Import necessary libraries
import os
import datetime
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ee
import geemap


In [2]:
# Better visualization
plt.style.use('ggplot')

In [3]:
EE_PROJECT_ID = os.environ.get('EE_PROJECT_ID', "clear-router-390022")


In [4]:
geemap.ee_initialize(project=EE_PROJECT_ID)

In [94]:
geojson_data = {"type":"Polygon","coordinates":[[[37.048302,-0.019913],[37.048302,0.513604],[37.514191,0.513604],[37.514191,-0.019913],[37.048302,-0.019913]]]}

# Area = 3080.12 km2

# Extract the coordinates
coordinates = geojson_data["coordinates"][0]

# Find the minimum and maximum longitude and latitude
min_lon = min([coord[0] for coord in coordinates])
max_lon = max([coord[0] for coord in coordinates])
min_lat = min([coord[1] for coord in coordinates])
max_lat = max([coord[1] for coord in coordinates])

# Construct the bounding box
bbox = [min_lon, min_lat, max_lon, max_lat]

print("bbox:", bbox)


bbox: [37.048302, -0.019913, 37.514191, 0.513604]


In [95]:
# Using ee.Geometry.Polygon
roi = ee.Geometry.Polygon(coordinates)
display(roi)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              37.048302,
              -0.019913
            ],
            [
              37.048302,
              0.513604
            ],
            [
              37.514191,
              0.513604
            ],
            [
              37.514191,
              -0.019913
            ],
            [
              37.048302,
              -0.019913
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [96]:
# Extract center coordinates
bbox_coords = geojson_data["coordinates"][0]
longitude = sum([coord[0] for coord in bbox_coords]) / len(bbox_coords)
latitude = sum([coord[1] for coord in bbox_coords]) / len(bbox_coords)
center = [latitude, longitude] 
display(center)

[0.1934938, 37.2346576]

In [50]:
start_date = '2024-05-31'
end_date = '2024-06-30'

In [64]:
# Load Sentinel-2 image collection
Kenya_IC = ee.ImageCollection('users/pengyuhao/FAO/GEO-AI_2024/Kenya_IC')

# Check the numbers of image in the collection
print("The number of available images is:", Kenya_IC.size().getInfo())


The number of available images is: 24


In [57]:
first = Kenya_IC.first()
print(Kenya_IC.first())


ee.Image({
  "functionInvocationValue": {
    "functionName": "Collection.first",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.load",
          "arguments": {
            "id": {
              "constantValue": "users/pengyuhao/FAO/GEO-AI_2024/Kenya_IC"
            }
          }
        }
      }
    }
  }
})


In [58]:
Kenya_s2_Collection = Kenya_IC.getInfo()

display(Kenya_s2_Collection)

{'type': 'ImageCollection',
 'bands': [],
 'version': 1720680956354189,
 'id': 'users/pengyuhao/FAO/GEO-AI_2024/Kenya_IC',
 'features': [{'type': 'Image',
   'bands': [{'id': 'blue_p50',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [2784, 2783],
     'crs': 'EPSG:4326',
     'crs_transform': [0.0001796630568239043,
      0,
      36.99998958537167,
      0,
      -0.0001796630568239043,
      0.5000022871409256]},
    {'id': 'green_p50',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [2784, 2783],
     'crs': 'EPSG:4326',
     'crs_transform': [0.0001796630568239043,
      0,
      36.99998958537167,
      0,
      -0.0001796630568239043,
      0.5000022871409256]},
    {'id': 'red_p50',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [2784, 2783],
     'crs': 'EPSG:4326',
     'crs_transform': [0.0001796630568239043,
      0,
      36.99998958537167,
      0,
      -0.000179663056

In [6]:
filteredKenya = Kenya_IC.filterDate('2024-05-31', '2024-06-30')
print(filteredKenya); 


ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.load",
          "arguments": {
            "id": {
              "constantValue": "users/pengyuhao/FAO/GEO-AI_2024/Kenya_IC"
            }
          }
        }
      },
      "filter": {
        "functionInvocationValue": {
          "functionName": "Filter.dateRangeContains",
          "arguments": {
            "leftValue": {
              "functionInvocationValue": {
                "functionName": "DateRange",
                "arguments": {
                  "end": {
                    "constantValue": "2024-06-30"
                  },
                  "start": {
                    "constantValue": "2024-05-31"
                  }
                }
              }
            },
            "rightField": {
              "constantValue": "system:time_start"
          

In [14]:
# Filter the collection for time range for image

Kenya_s2_Image = Kenya_IC.filterDate(start_date, end_date).filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 50).filterBounds(roi)

In [ ]:
monthlyKenya = Kenya_s2_Image.reduce(ee.Reducer.mean(), ee.Date.fromYMD(2024, 5, 30).advance(1, 'month'))
print(monthlyKenya); 

In [15]:
Kenya_s2_Image = monthlyKenya.mosaic().clip(roi)

In [97]:
# Create an interactive map
Map1 = geemap.Map(center=center, zoom=12)
Map1.add_basemap("ROADMAP")

["blue_p50", "green_p50", "red_p50", "re1_p50", "re2_p50", "re3_p50", "nir_p50", "nira_p50", "swir1_p50", "swir2_p50", "VV_p50", "VH_p50"]

In [98]:
# Visualize the image during the flood
s2_Viz = {"min":0, "max":3000, "bands":["red_p50", "green_p50", "blue_p50"]}
Map1.addLayer(Kenya_IC, s2_Viz, "Kenya_s2_Image")
Map1


Map(center=[0.1934938, 37.2346576], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

In [101]:
# Calculate NDVI
ndvi = first.normalizedDifference(['nir_p50', 'red_p50']).rename("ndvi")
ndvi

In [108]:
ndvi_Viz = {
    "min": 0,
    "max": 1,
    "palette": ["#d73027", "#ffffbf", "#1a9641"],
}

Map2 = geemap.Map(center=center, zoom=11)
Map2.addLayer(ndvi, ndvi_Viz, "NDVI")
Map2

Map(center=[0.1934938, 37.2346576], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…